
<img src="../images/starwarsstartrek.png" style="float: center; margin: 0px; height: 150px; width: 500px"> 

## SUBREDDIT CLASSIFIER

###### _Author: Afolabi Cardoso_

---

### Pre-Processing and Logistic Regression modelling

---
#### Contents:
[Overview](#Overview) | [Imports](#Imports) | [Data Cleaning Training Set](#Data-Cleaning-Training-Set) | [Data Cleaning Test Set](#Data-Cleaning-Test-Set) | [Exports](#Exports)


---
#### Overview



---
#### Imports

In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import plot_confusion_matrix

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

In [41]:
df = pd.read_csv('../data/preprocessed.csv')
df.head()

,title,selftext,created_utc,subreddit,mood
0,Star wars game,,1646329318,starwars,-0.5574
1,If a Star Wars game about the famous wars in c...,,1646329234,starwars,-0.8020
2,What makes a Star Wars fana FAN,,1646329180,starwars,-0.1449
3,What makes a Star Wars fan,,1646328985,starwars,-0.3182
4,Random Clone Wars Thoughts,The series makes me appreciate Anakin so much ...,1646328915,starwars,-0.5574


In [36]:
df.head()

,title,selftext,created_utc,subreddit
0,Star wars game,,1646329318,StarWars
1,If a Star Wars game about the famous wars in c...,,1646329234,StarWars
2,What makes a Star Wars fan...a FAN?,,1646329180,StarWars
3,What makes a Star Wars fan?,,1646328985,StarWars
4,Random Clone Wars Thoughts,The series makes me appreciate Anakin so much ...,1646328915,StarWars


#### Pre processing


Fot the pre processing, we will start by removing punctuations. I will use regex and pandas str.replace method

In [37]:
df['title'] = df['title'].str.replace('[^\w\s]','')

/var/folders/db/yz8g3qx147g7dfn7_8w2pxl80000gn/T/ipykernel_95184/2270917832.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace('[^\w\s]','')


In [38]:
df['title']

0                                          Star wars game
1       If a Star Wars game about the famous wars in c...
2                         What makes a Star Wars fana FAN
3                              What makes a Star Wars fan
4                              Random Clone Wars Thoughts
                              ...                        
1995      I have finally watched DS9 from start to finish
1996                                           TOS script
1997    How did you get into Trek What series was your...
1998    If youve never seen this this is the prototype...
1999    The Ready Room All In Sonequa MartinGreen amp ...
Name: title, Length: 2000, dtype: object